In [0]:
from pyspark.sql.types import StructType, StructField, DateType, StringType, IntegerType, FloatType,TimestampType,DateType
from pyspark.sql.functions import col, to_timestamp,count,lead,unix_timestamp, date_format,desc,to_date
from pyspark.sql.window import Window
from datetime import datetime

"""df = spark.read.format("csv") \
  .option("inferSchema", True) \
  .option("header", True) \
  .option("sep", ',') \
  .load(["dbfs:/FileStore/Sales_January_2019.csv","dbfs:/FileStore/Sales_February_2019.csv","dbfs:/FileStore/Sales_March_2019.csv"])
"""
df=spark.read.csv(path=["dbfs:/FileStore/Sales_January_2019.csv","dbfs:/FileStore/Sales_February_2019.csv","dbfs:/FileStore/Sales_March_2019.csv"],inferSchema=True,header=True,sep=',')



df.show()
df.printSchema()



+--------+--------------------+----------------+----------+--------------+--------------------+
|Order ID|             Product|Quantity Ordered|Price Each|    Order Date|    Purchase Address|
+--------+--------------------+----------------+----------+--------------+--------------------+
|  162009|              iPhone|               1|     700.0|03/28/19 20:59|942 Church St, Au...|
|  162009|Lightning Chargin...|               1|     14.95|03/28/19 20:59|942 Church St, Au...|
|  162009|    Wired Headphones|               2|     11.99|03/28/19 20:59|942 Church St, Au...|
|  162010|Bose SoundSport H...|               1|     99.99|03/17/19 05:39|261 10th St, San ...|
|  162011|34in Ultrawide Mo...|               1|    379.99|03/10/19 00:01|764 13th St, San ...|
|  162012|AA Batteries (4-p...|               1|      3.84|03/20/19 21:33|187 Ridge St, San...|
|  162013|34in Ultrawide Mo...|               1|    379.99|03/15/19 23:05|904 Main St, Aust...|
|  162014|USB-C Charging Cable|         

In [0]:
#1.	Cleanse the data removing blank rows 
print("rawcount=",df.count())

df2=df.dropna(how="any")

print("cleancount=",df2.count())


rawcount= 36985
cleancount= 36821


In [0]:
#CHanging column names 

df2=df2.withColumnRenamed('Quantity Ordered','Quantity')
df2=df2.withColumnRenamed('Order Date','Order_date')
df2=df2.withColumnRenamed('Price Each','Price')
df2=df2.withColumnRenamed('Order ID','Order_id')
df2=df2.withColumnRenamed('Purchase Address','Address')
df2 = df2.withColumn("Order_date", date_format(unix_timestamp(col("Order_date"), "MM/dd/yy HH:mm").cast("timestamp"), "MM-dd-yyyy"))

df2.show()
df2.printSchema()

+--------+--------------------+--------+------+----------+--------------------+
|Order_id|             Product|Quantity| Price|Order_date|             Address|
+--------+--------------------+--------+------+----------+--------------------+
|  162009|              iPhone|       1| 700.0|03-28-2019|942 Church St, Au...|
|  162009|Lightning Chargin...|       1| 14.95|03-28-2019|942 Church St, Au...|
|  162009|    Wired Headphones|       2| 11.99|03-28-2019|942 Church St, Au...|
|  162010|Bose SoundSport H...|       1| 99.99|03-17-2019|261 10th St, San ...|
|  162011|34in Ultrawide Mo...|       1|379.99|03-10-2019|764 13th St, San ...|
|  162012|AA Batteries (4-p...|       1|  3.84|03-20-2019|187 Ridge St, San...|
|  162013|34in Ultrawide Mo...|       1|379.99|03-15-2019|904 Main St, Aust...|
|  162014|USB-C Charging Cable|       1| 11.95|03-01-2019|10 13th St, San F...|
|  162015|AA Batteries (4-p...|       1|  3.84|03-02-2019|949 Jefferson St,...|
|  162016|AAA Batteries (4-...|       5|

In [0]:
#Add Sales Column

df2=df2.withColumn('sales',((df2.Quantity)*(df2.Price)))
df2.show()
df2.createOrReplaceTempView("orders")


+--------+--------------------+--------+------+----------+--------------------+------------------+
|Order_id|             Product|Quantity| Price|Order_date|             Address|             sales|
+--------+--------------------+--------+------+----------+--------------------+------------------+
|  162009|              iPhone|       1| 700.0|03-28-2019|942 Church St, Au...|             700.0|
|  162009|Lightning Chargin...|       1| 14.95|03-28-2019|942 Church St, Au...|             14.95|
|  162009|    Wired Headphones|       2| 11.99|03-28-2019|942 Church St, Au...|             23.98|
|  162010|Bose SoundSport H...|       1| 99.99|03-17-2019|261 10th St, San ...|             99.99|
|  162011|34in Ultrawide Mo...|       1|379.99|03-10-2019|764 13th St, San ...|            379.99|
|  162012|AA Batteries (4-p...|       1|  3.84|03-20-2019|187 Ridge St, San...|              3.84|
|  162013|34in Ultrawide Mo...|       1|379.99|03-15-2019|904 Main St, Aust...|            379.99|
|  162014|

In [0]:
#2.	Get the date on which max sales was done by product in these 3 months
max_sales_by_product=spark.sql("select Order_date,Product,sum(sales) as max_sales from orders group by Product,Order_date order by max_sales desc limit 1").show()

+----------+------------------+---------+
|Order_date|           Product|max_sales|
+----------+------------------+---------+
|03-18-2019|Macbook Pro Laptop|  32300.0|
+----------+------------------+---------+



In [0]:
#3.	Get the date on which max sales was done for all products in these 3 months
max_sales_date=spark.sql("select Order_date,sum(sales) as max_sales from orders group by Order_date order by max_sales desc limit 1").show()

+----------+------------------+
|Order_date|         max_sales|
+----------+------------------+
|03-22-2019|102992.10999999993|
+----------+------------------+



In [0]:
#4.	Get the average sales value for each product in these 3 months
avg_sales_product=spark.sql("select Product,avg(sales) as max_sales from orders group by Product order by max_sales desc").show()

+--------------------+------------------+
|             Product|         max_sales|
+--------------------+------------------+
|  Macbook Pro Laptop|            1700.0|
|     ThinkPad Laptop|1001.1847311827918|
|              iPhone| 700.5166051660517|
|        Google Phone| 601.5530629853322|
|            LG Dryer|             600.0|
|  LG Washing Machine|             600.0|
|     Vareebadd Phone| 400.9367681498829|
|27in 4K Gaming Mo...| 390.9521463815789|
|34in Ultrawide Mo...|381.20402555910533|
|       Flatscreen TV| 301.3114754098361|
|Apple Airpods Hea...|150.76628352490422|
|    27in FHD Monitor|150.69182486631038|
|        20in Monitor|110.69326086956568|
|Bose SoundSport H...|101.00000000000091|
|Lightning Chargin...|16.038626660341954|
|USB-C Charging Cable|13.095375982042984|
|    Wired Headphones| 13.03612911939485|
|AA Batteries (4-p...| 5.174857142857234|
|AAA Batteries (4-...| 4.464702542581957|
+--------------------+------------------+



In [0]:
#5.	Create a combined dataset merging all these 3 datasets with order by date in desc order and add a new column which is “salesdiff” where this column will contain the difference of the sales in the current row (current date of that row) and the next row (previous date of that row, as the date columns are sorted by desc) grouped on the product. For the last row, next row will be blank so consider the sales as 0

sales_diff=spark.sql("select *,ABS((lead(sales,1) over( order by Order_date desc))-sales) as salesdiff from orders").show()


+--------+--------------------+--------+------+----------+--------------------+------+------------------+
|Order_id|             Product|Quantity| Price|Order_date|             Address| sales|         salesdiff|
+--------+--------------------+--------+------+----------+--------------------+------+------------------+
|  162358|       Flatscreen TV|       1| 300.0|04-01-2019|444 12th St, New ...| 300.0|            294.02|
|  162541|AAA Batteries (4-...|       2|  2.99|04-01-2019|672 2nd St, Atlan...|  5.98|            374.01|
|  165007|34in Ultrawide Mo...|       1|379.99|04-01-2019|389 Willow St, Ne...|379.99|            365.04|
|  165622|Lightning Chargin...|       1| 14.95|04-01-2019|65 Madison St, Bo...| 14.95|               0.0|
|  166250|Lightning Chargin...|       1| 14.95|04-01-2019|159 Walnut St, Lo...| 14.95|             11.11|
|  166309|AA Batteries (4-p...|       1|  3.84|04-01-2019|2 Church St, Seat...|  3.84|376.15000000000003|
|  166567|34in Ultrawide Mo...|       1|379.99

In [0]:
#6.	Get the orderId and purchase address details who made max sales in all the 3 months

max_purchase_order=spark.sql("select Order_id,Address,max(sales) as max_sales from orders group by Order_id,Address order by max_sales desc limit 1 ").show()

+--------+--------------------+---------+
|Order_id|             Address|max_sales|
+--------+--------------------+---------+
|  149611|850 6th St, New Y...|  1999.98|
+--------+--------------------+---------+



In [0]:
#7:	Extract city from the purchase address column which is 2nd element in , delimited separated string and determine the city from where more orders came in all these 3 months

from pyspark.sql.functions import split, col, count, desc

df_city2=spark.sql("SELECT SUBSTRING_INDEX(SUBSTRING_INDEX(Address, ',', 2), ',', -1) AS City FROM Orders").createOrReplaceTempView("OrdersWithCity")


address_city_df=spark.sql("select * from OrdersWithCity")

#address_city_df.show()

Max_orders_from_city_df=spark.sql("select city,count(*) as No_of_orders from OrdersWithCity group by city order by No_of_orders desc limit 1 ").show()


+--------------+------------+
|          city|No_of_orders|
+--------------+------------+
| San Francisco|        8863|
+--------------+------------+



In [0]:
#8.	Get the total order count details for each city in all the 3 months

Total_orders_from_cities_df=spark.sql("select city,count(*) as No_of_orders from OrdersWithCity where city IS NOT NULL group by city order by No_of_orders desc").show()


+--------------+------------+
|          city|No_of_orders|
+--------------+------------+
| San Francisco|        8863|
|   Los Angeles|        5886|
| New York City|        4895|
|        Boston|        3952|
|        Dallas|        2968|
|       Atlanta|        2967|
|       Seattle|        2855|
|      Portland|        2442|
|        Austin|        1993|
+--------------+------------+

